# Flowsheet Validator

The overall configuration of a flowsheet can be specified without providing detail within the individual process groups. This is demonstrated in Chapter 4 of the d'Anterroches (2006) thesis where upper case alphabetic characters substitute for process groups. The resulting strings do not form valid SFILES sequences, but are nevertheless useful means of describing flowsheets at a higher level of abstraction.

The following examples are from d'Anterroches (2006):

     AC[<B]DE                                  <== Figure 4.7
     ABC[<J<I<H]D[<L[<K]<M]EFG                 <== Figure 4.8 (w/correction)
     HIJC[<B<A]D[<L[<K]<M]EFG                  <== Figure 4.8 (alternative)
     AB<1CD1E                                  <== Figure 4.9
     OA<1E[P]F[Q]G<2MN[R]J<3K[IH3]L[S]D2CB1    <== Figure 4.10

The following cell demonstrates the parsing and validation of these sequences using the `pyparsing` library.

In [208]:
from pyparsing import Literal, Word, Group
from pyparsing import Optional, OneOrMore, ZeroOrMore
from pyparsing import alphas, nums
from pyparsing import nestedExpr

GT = Literal(">")
LT = Literal("<")

# process group
processgroup = Word(alphas.upper(), exact=1)
           
# process sequence is comprised of connectors, process group, and recycles                                             
connector = Optional(GT | LT, GT)
recycle = Word(nums, exact=1)
sequence = Group(OneOrMore(connector + (processgroup | recycle )))

# nested branches
branchsequence = Group(OneOrMore(connector + (processgroup | recycle )))
branch = nestedExpr(opener="[", closer="]", content=branchsequence)
branch.setName('branch')

# sfiles expression
sfiles = sequence + ZeroOrMore(branch | sequence)

At this stage, `sfiles` is an object from the [`ParserElement`](https://pythonhosted.org/pyparsing/pyparsing.ParserElement-class.html) class from the `pyparsing` library.

In [214]:
tests = """\
    AC[<B]DE
    ABC[<J<I<H]D[<L[<K]<M]EFG
    HIJC[<B<A]D[<L[<K]<M]EFG
    AB<1CD1E
    OA<1E[P]F[Q]G<2MN[R]J<3K[IH3]L[S]D2CB1
"""

result = sfiles.runTests(tests)


AC[<B]DE
[[">", 'A', ">", 'C'], [['<', 'B']], [">", 'D', ">", 'E']]
[0]:
  [">", 'A', ">", 'C']
[1]:
  [['<', 'B']]
  [0]:
    ['<', 'B']
[2]:
  [">", 'D', ">", 'E']


ABC[<J<I<H]D[<L[<K]<M]EFG
[[">", 'A', ">", 'B', ">", 'C'], [['<', 'J', '<', 'I', '<', 'H']], [">", 'D'], [['<', 'L'], [['<', 'K']], ['<', 'M']], [">", 'E', ">", 'F', ">", 'G']]
[0]:
  [">", 'A', ">", 'B', ">", 'C']
[1]:
  [['<', 'J', '<', 'I', '<', 'H']]
  [0]:
    ['<', 'J', '<', 'I', '<', 'H']
[2]:
  [">", 'D']
[3]:
  [['<', 'L'], [['<', 'K']], ['<', 'M']]
  [0]:
    ['<', 'L']
  [1]:
    [['<', 'K']]
    [0]:
      ['<', 'K']
  [2]:
    ['<', 'M']
[4]:
  [">", 'E', ">", 'F', ">", 'G']


HIJC[<B<A]D[<L[<K]<M]EFG
[[">", 'H', ">", 'I', ">", 'J', ">", 'C'], [['<', 'B', '<', 'A']], [">", 'D'], [['<', 'L'], [['<', 'K']], ['<', 'M']], [">", 'E', ">", 'F', ">", 'G']]
[0]:
  [">", 'H', ">", 'I', ">", 'J', ">", 'C']
[1]:
  [['<', 'B', '<', 'A']]
  [0]:
    ['<', 'B', '<', 'A']
[2]:
  [">", 'D']
[3]:
  [['<', 'L'], [['<', 'K']]